# Benin Electrification Analysis

This notebook runs the least-cost electrification model on Benin settlement data.

Results:
- Technology selection per settlement
- Investment requirements
- Summary statistics and visualizations


In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path("..").resolve()))

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

from benin_least_cost.schema import DataValidator, DataSchema as DS
from benin_least_cost.parameters import ProjectConfig
from benin_least_cost.demand import run_demand_model
from benin_least_cost.lcoe import run_lcoe_model


## Load Data


In [ ]:
gdf = gpd.read_file("../data/settlements.geojson")
gdf = DataValidator.validate_input(gdf)

print(f"Loaded {len(gdf)} settlements")
print(f"Columns: {len(gdf.columns)}")


## Run Model


In [ ]:
config = ProjectConfig()

gdf = run_demand_model(gdf, config)
gdf = run_lcoe_model(gdf, config)

print("Model complete")


## Results


In [ ]:
tech_counts = gdf[DS.OPTIMAL_TECH].value_counts()
total_inv = gdf[DS.INVESTMENT].sum()

print("Technology Selection:")
print(tech_counts)
print(f"\nTotal Investment: ${total_inv:,.0f}")


In [ ]:
gdf[[DS.PROJECTED_DEMAND, DS.PROJECTED_PEAK, DS.LCOE_GRID, DS.LCOE_MG, DS.LCOE_SHS, DS.INVESTMENT]].describe()


## Visualizations


In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
gdf[DS.OPTIMAL_TECH].value_counts().plot(kind='bar', ax=ax)
ax.set_title('Technology Selection')
ax.set_xlabel('Technology')
ax.set_ylabel('Number of Settlements')
plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, col, title in zip(axes, [DS.LCOE_GRID, DS.LCOE_MG, DS.LCOE_SHS], ['Grid', 'Mini-Grid', 'SHS']):
    gdf[col].clip(upper=2).hist(bins=50, ax=ax)
    ax.set_title(f'{title} LCOE')
    ax.set_xlabel('USD/kWh')
plt.tight_layout()
plt.show()


## Save Results


In [ ]:
output_path = Path("../results.geojson")
gdf.to_file(output_path, driver="GeoJSON")
print(f"Saved to {output_path}")
